In [1]:
import json

with open('./data/bdd100k/labels/det_20/det_train.json') as f:
    data = json.load(f)

In [44]:
import os 

image_files = os.listdir('./data/bdd100k/images/10k/train')

image_files

['0004a4c0-d4dff0ad.jpg',
 '00054602-3bf57337.jpg',
 '00067cfb-e535423e.jpg',
 '00091078-59817bb0.jpg',
 '0010bf16-a457685b.jpg',
 '001b428f-059bac33.jpg',
 '001c2a14-c7138401.jpg',
 '0024b742-acbefa1a.jpg',
 '0027eed2-09c90000.jpg',
 '0027eed2-09c90001.jpg',
 '0027eed2-60fb0001.jpg',
 '0027eed2-815a0000.jpg',
 '0027eed2-815a0001.jpg',
 '0027eed2-a6630000.jpg',
 '0027eed2-a6630001.jpg',
 '002a3213-ab7f6730.jpg',
 '002bf392-acddbad2.jpg',
 '002d467e-00000000.jpg',
 '003571f2-f2848ebd.jpg',
 '003ddde7-36ee8299.jpg',
 '004071a4-049be89b.jpg',
 '00495359-1d04dd8a.jpg',
 '004af474-00000000.jpg',
 '004fd763-ad4c2b9c.jpg',
 '005bb196-00000000.jpg',
 '005c4fd3-cb4d49cf.jpg',
 '0066b72f-974f6883.jpg',
 '006a7635-c42f9f97.jpg',
 '0096bcca-c2027ec4.jpg',
 '00a360bd-27ccb1dd.jpg',
 '00a395fe-d60c0b47.jpg',
 '00a7ef03-00000000.jpg',
 '00a9cd6b-b39be004.jpg',
 '00aad4a0-ee8135fe.jpg',
 '00ad8a92-c4851839.jpg',
 '00bc0319-94afabc2.jpg',
 '00cc68dd-3d50a55f.jpg',
 '00d1c9e3-a7a7075f.jpg',
 '00d79c0a-2

In [19]:
train_matches = []

for det in data:
    if det.get('name') in image_files:
        print(f'Found match for: {det.get("name")}')
        train_matches.append((det, det.get("attributes").get("scene")))

Found match for: 00054602-3bf57337.jpg
Found match for: 00067cfb-e535423e.jpg
Found match for: 00091078-59817bb0.jpg
Found match for: 0010bf16-a457685b.jpg
Found match for: 00495359-1d04dd8a.jpg
Found match for: 0066b72f-974f6883.jpg
Found match for: 006a7635-c42f9f97.jpg
Found match for: 0096bcca-c2027ec4.jpg
Found match for: 00a395fe-d60c0b47.jpg
Found match for: 00a9cd6b-b39be004.jpg
Found match for: 00ad8a92-c4851839.jpg
Found match for: 00d79c0a-23bea078.jpg
Found match for: 00d8944b-e157478b.jpg
Found match for: 00de601c-858a8a8d.jpg
Found match for: 00f7eac8-5193d600.jpg
Found match for: 010fc651-19922861.jpg
Found match for: 01118704-2d838d7f.jpg
Found match for: 011d593a-89158b9d.jpg
Found match for: 012a9c41-edad898c.jpg
Found match for: 012e0351-723ac360.jpg
Found match for: 013ae324-36be6bc2.jpg
Found match for: 014474aa-accf35e1.jpg
Found match for: 01688cf2-237fbd49.jpg
Found match for: 01818f85-6397dd75.jpg
Found match for: 01a4933a-252e566a.jpg
Found match for: 01c231b4

In [20]:
import json

with open('./data/bdd100k/labels/det_20/det_val.json') as f:
    val_data = json.load(f)
    
val_image_files = os.listdir('./data/bdd100k/images/10k/val')

In [45]:
val_matches = []

for det in val_data:
    if det.get('name') in image_files:
        print(f'Found match for: {det.get("name"), det.get("attributes").get("scene")}')
        val_matches.append((det.get("name"), det.get("attributes").get("scene")))

Found match for: ('b1d0091f-f2c2d2ae.jpg', 'highway')
Found match for: ('b1d0a191-5490450b.jpg', 'residential')
Found match for: ('b1d10d08-c35503b8.jpg', 'city street')
Found match for: ('b1d971b4-ac67ca0d.jpg', 'residential')
Found match for: ('b1e1a7b8-a7426a97.jpg', 'highway')
Found match for: ('b1e8ad72-c3c79240.jpg', 'highway')
Found match for: ('b1eb9133-5cc75c18.jpg', 'highway')
Found match for: ('b1f0efd9-37a14dda.jpg', 'residential')
Found match for: ('b1f4491b-d8d1459c.jpg', 'city street')
Found match for: ('b1f85377-44885085.jpg', 'highway')
Found match for: ('b20b69d2-650e674d.jpg', 'residential')
Found match for: ('b21c86ac-0dc77d82.jpg', 'city street')
Found match for: ('b222c329-5dc8dbf7.jpg', 'city street')
Found match for: ('b24f7455-e8c55d6a.jpg', 'city street')
Found match for: ('b251064f-e7a165fd.jpg', 'city street')
Found match for: ('b2694461-db4f4470.jpg', 'highway')
Found match for: ('b26f8762-61cfb033.jpg', 'parking lot')
Found match for: ('b280c9c1-a75d23e5.j

In [1]:
for match_i in val_matches:
    for match_j in train_matches:
        if match_i[0] == match_j[0]:
            print(f'Found duplicate for: {match_i[0]} and {match_j[0]}')

NameError: name 'val_matches' is not defined

In [25]:
# count the number of  images in each scene category
scene_counts = {}
for match in train_matches:
    scene = match[1]
    if scene in scene_counts:
        scene_counts[scene] += 1
    else:
        scene_counts[scene] = 1
        
scene_counts

{'city street': 1756,
 'parking lot': 23,
 'highway': 708,
 'undefined': 12,
 'residential': 467,
 'tunnel': 5,
 'gas stations': 1}

In [29]:
# count city street scenes vs all other scenes
city_street_count = 0
non_city_street_count = 0
for scene in scene_counts:
    if 'city street' in scene:
        city_street_count += scene_counts[scene]
        
    else:
        non_city_street_count += scene_counts[scene]
        
city_street_count, non_city_street_count

(254, 200)

In [28]:
scene_counts = {}
for match in val_matches:
    scene = match[1]
    if scene in scene_counts:
        scene_counts[scene] += 1
    else:
        scene_counts[scene] = 1
        
scene_counts

{'highway': 112,
 'residential': 85,
 'city street': 254,
 'parking lot': 2,
 'tunnel': 1}